In [1]:
import telebot
from telebot import types
import requests
import wget

import pandas as pd
import sqlite3
import os


TOKEN = "7072729916:AAE-bUOrmJDJuJuOdyGNiB58fYWgOC5q6QU"
bot = telebot.TeleBot(TOKEN)

s_user = "Обычный пользователь"
s_admin = "Администратор"
s_back = "Назад"
s_exit = "Выход"
s_pass_change = "Сменить пароль"

@bot.message_handler(commands=['start', 'hello']) # приветствие
def send_welcome(message):

    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    but1 = types.KeyboardButton(s_admin)
    but2 = types.KeyboardButton(s_user)
    markup.add(but1, but2)

    bot.reply_to(message, "Здравствуй, " + message.chat.first_name + "! Кто ты?", parse_mode='html', reply_markup=markup)


@bot.message_handler(content_types=['text'])
def autorization(message):
    if message.text == s_admin:
        markup = types.ReplyKeyboardRemove()
        sent = bot.send_message(message.chat.id, 'Введите пароль:', reply_markup=markup)
        bot.register_next_step_handler(sent, login)

    elif message.text == 'Загрузить таблицу':
        markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
        exit = types.KeyboardButton(s_exit)
        back = types.KeyboardButton(s_back)
        markup.add(exit, back)
        sent = bot.send_message(message.chat.id, 'Загрузите таблицу')
        bot.register_next_step_handler(sent, get_file)

    elif message.text == s_back:
        markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
        but1 = types.KeyboardButton(s_admin)
        but2 = types.KeyboardButton(s_user)
        markup.add(but1, but2)
        bot.send_message(message.chat.id, 'Кто ты?', reply_markup=markup)

    elif message.text == s_exit:
        markup = types.ReplyKeyboardRemove()
        bot.send_message(message.chat.id, 'Всего хорошего', reply_markup=markup)

    elif message.text == s_user:
        markup = types.ReplyKeyboardRemove()
        bot.send_message(message.chat.id, "Отлично!", reply_markup=markup)

    elif message.text == s_pass_change:
       markup = types.ReplyKeyboardRemove()
       sent = bot.send_message(message.chat.id, "Введите новый пароль", reply_markup=markup)
       bot.register_next_step_handler(sent, pass_writer)

def login(message):
    password = pass_reader()
    if message.text == password:
        markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
        exit = types.KeyboardButton(s_exit)
        back = types.KeyboardButton(s_back)
        pass_change = types.KeyboardButton(s_pass_change)
        load_table = types.KeyboardButton('Загрузить таблицу')
        markup.add(exit, back, pass_change, load_table)
        bot.send_message(message.chat.id, 'Вход успешен', reply_markup=markup)
    else:
        bot.send_message(message.chat.id, 'Пароль неверен')
        markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
        but1 = types.KeyboardButton(s_admin)
        but2 = types.KeyboardButton(s_user)
        markup.add(but1, but2)
        bot.send_message(message.chat.id, 'Кто же ты на самом деле?', reply_markup=markup)

def get_file(message):
  try:

    URL = bot.get_file_url(message.document.file_id)  # получаем ссылку на документ
    URL_parts = URL.split(".") # дробим ссылку по точкам, чтобы последним элементом списка было расширение файла

    if URL_parts[-1] == 'xlsx': # если расширение xlsx -- бот загружает таблицу
      bot.send_message(message.chat.id, "Таблица загружена")
      folder_path = os.getcwd()
      excel_file = os.path.join(folder_path, 'data.xlsx')

      if os.path.exists(excel_file): # для исключения создания копий, удаляем старую версию БД, если такая существует
         os.remove(excel_file)

      response = wget.download(URL, 'data.xlsx')
      convertation('data.xlsx')

    else: # если расширение НЕ xlsx -- бот пишет что недопустимый тип файла
      bot.send_message(message.chat.id, "Недопустимый тип файла. Загрузите таблицу в формате .xlsx")

  except: # если загруженный файл -- не документ, то сразу посылаем куда подальше
    bot.send_message(message.chat.id, 'Загруженный элемент не является файлом')




def convertation(file_name):
  folder_path = os.getcwd() # путь к корневой папке
  excel_file = os.path.join(folder_path, file_name) # добавляем путь к файлу
  if os.path.exists(excel_file): # проверяем, что файл по указанному пути существует
    xlsx = pd.ExcelFile(excel_file) # переводим xlsx в pd формат (DataFrame object)
    sheet_name = 'list1'
    db = xlsx.parse(sheet_name)
    db_file = os.path.join(folder_path, 'data.db') # указываем путь для сохранения нового файла
    conn = sqlite3.connect(db_file) # создание подключения к базе данных
    cur = conn.cursor() # создание объекта "курсор" 
    db.to_sql('Table', conn, index = False) # , if_exist = 'replace' # собственно конвертация из DataFrame в SQL 
    conn.commit() # сохранение изменений
    conn.close() # закрытие соединения

  else: 
      print(f"Excel-файл '{excel_file}' не существует") # иначе говорим, что Excel файла не существует


def pass_reader():
   f = open('password.txt','r')
   str = f.readline()
   return str

def pass_writer(message):
   password = message.text
   f = open('password.txt','w')
   f.truncate(0)
   f.write(password)

   markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
   exit = types.KeyboardButton(s_exit)
   back = types.KeyboardButton(s_back)
   pass_change = types.KeyboardButton(s_pass_change)
   load_table = types.KeyboardButton('Загрузить таблицу')
   markup.add(exit, back, pass_change, load_table)
   bot.send_message(message.chat.id, 'Пароль изменен!', reply_markup=markup)




bot.polling(non_stop=True)